In [1]:
import torch
import torchvision
import fastai
import os
import pandas as pd
import shutil
import random
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchvision.transforms as transforms
from PIL import Image
import pydicom
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import cv2

In [ ]:
csv_path = '/kaggle/input/iaaa-mri-challenge/train.csv'
df = pd.read_csv(csv_path)
df.head()

In [ ]:
os.mkdir('/kaggle/working/separated_data')


In [ ]:
separated_data_folder = '/kaggle/working/separated_data'
data_folder = '/kaggle/input/iaaa-mri-challenge/data'

normal_folder = os.path.join(separated_data_folder, 'normal')
abnormal_folder = os.path.join(separated_data_folder, 'abnormal')

os.makedirs(normal_folder, exist_ok=True)
os.makedirs(abnormal_folder, exist_ok=True)

# Iterate through the CSV file and move folders
for index, row in df.iterrows():
    series_uid = row['SeriesInstanceUID']
    prediction = row['prediction']
    
    # Determine the source and destination paths
    src_folder = os.path.join(data_folder, series_uid)
    if prediction == 0:
        dst_folder = os.path.join(normal_folder, series_uid)
    else:
        dst_folder = os.path.join(abnormal_folder, series_uid)
    
    # Move the folder
    if os.path.exists(src_folder):
        shutil.copytree(src_folder, dst_folder)
    else:
        print(f"Folder {src_folder} does not exist")

print("Folders separated successfully!")

In [ ]:
os.mkdir('/kaggle/working/splited_dataset')

In [ ]:
# Paths to the normal and abnormal folders

base_folder = '/kaggle/working/splited_dataset'

# Split ratios
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# Create directories for train, val, and test sets
train_folder = os.path.join(base_folder, 'train')
val_folder = os.path.join(base_folder, 'val')
test_folder = os.path.join(base_folder, 'test')

for subset in ['train', 'val', 'test']:
    os.makedirs(os.path.join(train_folder, 'normal'), exist_ok=True)
    os.makedirs(os.path.join(train_folder, 'abnormal'), exist_ok=True)
    os.makedirs(os.path.join(val_folder, 'normal'), exist_ok=True)
    os.makedirs(os.path.join(val_folder, 'abnormal'), exist_ok=True)
    os.makedirs(os.path.join(test_folder, 'normal'), exist_ok=True)
    os.makedirs(os.path.join(test_folder, 'abnormal'), exist_ok=True)

# Function to split and move folders
def split_and_move_folders(src_folder, dst_base_folder):
    folders = [f for f in os.listdir(src_folder) if os.path.isdir(os.path.join(src_folder, f))]
    random.shuffle(folders)
    
    train_count = int(len(folders) * train_ratio)
    val_count = int(len(folders) * val_ratio)
    test_count = len(folders) - train_count - val_count
    
    splits = {
        'train': folders[:train_count],
        'val': folders[train_count:train_count + val_count],
        'test': folders[train_count + val_count:]
    }
    
    for split, split_folders in splits.items():
        for folder in split_folders:
            src_path = os.path.join(src_folder, folder)
            dst_path = os.path.join(dst_base_folder, split, os.path.basename(src_folder), folder)
            shutil.move(src_path, dst_path)

# Split and move normal and abnormal folders
split_and_move_folders(normal_folder, base_folder)
split_and_move_folders(abnormal_folder, base_folder)

print("Data split into train, val, and test sets successfully!")

In [ ]:
os.mkdir('/kaggle/working/final_data')

In [ ]:
def process_and_save_dicom(src_folder, dst_folder):
    for root, _, files in os.walk(src_folder):
        for file in files:
            if file.endswith(".dcm"):
                dicom_path = os.path.join(root, file)
                dicom_data = pydicom.dcmread(dicom_path)
                
                # Extract metadata
                slice_orientation = dicom_data[(0x2001, 0x100b)].value
                if slice_orientation != 'SAGITTAL':
                    pixel_array = dicom_data.pixel_array
                    protocol_name = dicom_data.ProtocolName.replace(" ", "_")  # Remove spaces in description
                    instance_number = dicom_data.InstanceNumber

                    # Get the original folder name (parent of current root)
                    original_folder_name = os.path.basename(root)

                    # Create a new folder for the subject using original folder name + PatientID
                    subject_folder_name = f"{original_folder_name}"
                    subject_folder = os.path.join(dst_folder, subject_folder_name)
                    os.makedirs(subject_folder, exist_ok=True)

                    # Construct the save path with instance number in the filename
                    save_path = os.path.join(subject_folder, f"{os.path.splitext(file)[0]}_{protocol_name}_Instance{instance_number}.npy")

                    # Save the numpy array
              
                    np.save(save_path, pixel_array)

In [ ]:
output_folder = '/kaggle/working/final_data'

# Create output directories for train, val, and test sets
for subset in ['train', 'val', 'test']:
    for category in ['normal', 'abnormal']:
        os.makedirs(os.path.join(output_folder, subset, category), exist_ok=True)

# Process and save DICOM files in train, val, and test sets
for subset in ['train', 'val', 'test']:
    for category in ['normal', 'abnormal']:
        src_folder = os.path.join(base_folder, subset, category)
        dst_folder = os.path.join(output_folder, subset, category)
        process_and_save_dicom(src_folder, dst_folder)

print("DICOM files processed and saved as NumPy arrays with SeriesDescription in filenames.")

In [ ]:
def extract_instance_number(filename):
    """Extract the instance number from the filename."""
    # Assuming the instance number is always preceded by 'Instance'
    base_name = os.path.splitext(os.path.basename(filename))[0]
    instance_str = base_name.split('_')[-1]  # Get the last part after splitting by '_'
    
    # Remove any non-digit characters from the instance part to extract the number
    instance_number = ''.join(filter(str.isdigit, instance_str))
    
    return int(instance_number)

def show_all_slices_of_random_subject(output_folder):
    # List all subject folders in the output directory
    subject_folders = [f.path for f in os.scandir(output_folder) if f.is_dir()]
    
    if not subject_folders:
        print("No subjects found in the output folder.")
        return
    
    # Select a random subject folder
    random_subject_folder = random.choice(subject_folders)
    print(f"Displaying all slices for subject: {os.path.basename(random_subject_folder)}")
    
    # Get all .npy files in the subject's folder
    npy_files = [os.path.join(random_subject_folder, f) for f in os.listdir(random_subject_folder) if f.endswith(".npy")]
    
    if not npy_files:
        print("No .npy files found for the selected subject.")
        return
    
    # Sort files based on extracted instance numbers
    npy_files = sorted(npy_files, key=extract_instance_number)

    # Number of slices
    num_slices = len(npy_files)
    
    # Determine grid size (rows and columns) for displaying the slices
    num_columns = 5  # Display 5 images per row
    num_rows = (num_slices + num_columns - 1) // num_columns  # Calculate the number of rows needed
    
    # Create a figure with the calculated grid size
    fig, axes = plt.subplots(num_rows, num_columns, figsize=(15, num_rows * 3))
    axes = axes.flatten()  # Flatten the axes array
    
    # Display each slice in the grid
    for i, file_path in enumerate(npy_files):
        # Load the image data from the .npy file
        image_data = np.load(file_path)
        
        # Extract the instance number using the defined function
        instance_number = extract_instance_number(file_path)
        
        # Display the image and set the title as the instance number
        axes[i].imshow(image_data, cmap='gray')
        axes[i].set_title(f"Instance {instance_number}")
        axes[i].axis('off')
    
    # Hide any unused subplots
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Specify the base output folder (adjust based on your folder structure)
normal_folder = '/kaggle/working/final_data/train/normal'  # Change this to the relevant path

# Show a random subject's images
show_all_slices_of_random_subject(normal_folder)

In [ ]:
# Specify the base output folder (adjust based on your folder structure)
abnormal_folder = '/kaggle/working/final_data/train/abnormal'  # Change this to the relevant path

# Show a random subject's images
show_all_slices_of_random_subject(abnormal_folder)

In [ ]:
def process_subject_slices(subject_folder):
    """Process the slices of a subject, resize them, split into hemispheres, and return the array."""
    
    # Get all .npy files in the subject's folder
    npy_files = [os.path.join(subject_folder, f) for f in os.listdir(subject_folder) if f.endswith(".npy")]
    
    if not npy_files:
        print("No .npy files found in the selected subject folder.")
        return None
    
    # Sort files based on extracted instance numbers
    npy_files = sorted(npy_files, key=extract_instance_number)
    
    # Limit to the first 16 slices, or pad with zeros if there are fewer
    num_slices = 16
    processed_slices = []
    
    for i in range(num_slices):
        if i < len(npy_files):
            image_data = np.load(npy_files[i])
        else:
            # Create a zero array if there are not enough slices
            image_data = np.zeros((512, 512))  # Assuming original slices are 512x512
        
        # Resize to 256x256
        resized_slice = cv2.resize(image_data, (256, 256), interpolation=cv2.INTER_AREA)
        
        # Split into left and right hemispheres
        left_hemisphere = resized_slice[:, :128]
        right_hemisphere = resized_slice[:, 128:]
        
        # Append hemispheres to the processed_slices list
        processed_slices.append([left_hemisphere, right_hemisphere])
    
    # Convert to numpy array with shape (16, 2, 256, 128)
    processed_slices = np.array(processed_slices)
    
    return processed_slices

In [ ]:
def show_processed_slices(processed_slices):
    """Display the processed slices with left and right hemispheres."""
    if processed_slices is None:
        print("No slices to display.")
        return
    
    num_slices = processed_slices.shape[0]
    fig, axes = plt.subplots(num_slices, 2, figsize=(10, num_slices * 2.5))
    
    for i in range(num_slices):
        # Display left hemisphere
        axes[i, 0].imshow(processed_slices[i, 0], cmap='gray')
        axes[i, 0].set_title(f'Slice {i+1} - Left Hemisphere')
        axes[i, 0].axis('off')
        
        # Display right hemisphere
        axes[i, 1].imshow(processed_slices[i, 1], cmap='gray')
        axes[i, 1].set_title(f'Slice {i+1} - Right Hemisphere')
        axes[i, 1].axis('off')
    
    plt.tight_layout()
    plt.show()


In [ ]:
subject_folder = '/kaggle/working/final_data/train/abnormal/1.3.46.670589.11.10042.5.0.6596.2024012113454106397'  # Adjust the path as needed
processed_slices = process_subject_slices(subject_folder)

In [ ]:
# Show the processed slices
show_processed_slices(processed_slices)

In [ ]:
def plot_histograms_for_subject(subject_folder):
    # Get all .npy files in the subject's folder
    npy_files = [os.path.join(subject_folder, f) for f in os.listdir(subject_folder) if f.endswith(".npy")]
    
    if not npy_files:
        print("No .npy files found for the selected subject.")
        return
    
    # Sort files to maintain order (assuming filenames include instance numbers)
    npy_files.sort(key=lambda x: int(os.path.splitext(os.path.basename(x))[0].split('_')[-1].replace("Instance", "")))
    
    for i, file_path in enumerate(npy_files):
        # Load the image data from the .npy file
        image_data = np.load(file_path)
        
        # Resize the image to 256x256
        resized_image = cv2.resize(image_data, (256, 256), interpolation=cv2.INTER_LINEAR)
        
        # Split the image into left and right hemispheres
        left_hemisphere = resized_image[:, :128]
        right_hemisphere = resized_image[:, 128:]
        
        # Exclude zeros from histograms
        left_hemisphere_nonzero = left_hemisphere[left_hemisphere > 0]
        right_hemisphere_nonzero = right_hemisphere[right_hemisphere > 0]
        
        # Plot histograms
        fig, axes = plt.subplots(1, 2, figsize=(12, 4))
        
        if left_hemisphere_nonzero.size > 0:  # Ensure there is non-zero data to plot
            axes[0].hist(left_hemisphere_nonzero.ravel(), bins=255, color='blue', alpha=0.7)
        axes[0].set_title(f'Slice {i+1} - Left Hemisphere')
        axes[0].set_xlabel('Pixel Intensity')
        axes[0].set_ylabel('Frequency')

        if right_hemisphere_nonzero.size > 0:  # Ensure there is non-zero data to plot
            axes[1].hist(right_hemisphere_nonzero.ravel(), bins=255, color='green', alpha=0.7)
        axes[1].set_title(f'Slice {i+1} - Right Hemisphere')
        axes[1].set_xlabel('Pixel Intensity')
        axes[1].set_ylabel('Frequency')

        plt.suptitle(f"Histograms for Slice {i+1} of Subject {os.path.basename(subject_folder)}")
        plt.show()

In [ ]:
abnormal_folder = '/kaggle/working/final_data/train/abnormal/1.3.46.670589.11.10042.5.0.6596.2024012113454106397'  # Replace with the actual path to the subject's folder
plot_histograms_for_subject(abnormal_folder)

In [ ]:
normal_folder = '/kaggle/working/final_data/train/normal/1.3.46.670589.11.10042.5.0.6596.2024012100305431405'  # Replace with the actual path to the subject's folder
plot_histograms_for_subject(normal_folder)

In [2]:
def compute_histograms_for_subject(subject_folder, num_bins=255):
    # Get all .npy files in the subject's folder
    npy_files = [os.path.join(subject_folder, f) for f in os.listdir(subject_folder) if f.endswith(".npy")]
    
    if not npy_files:
        print(f"No .npy files found for the subject folder: {subject_folder}")
        return None
    
    # Sort files to maintain order (assuming filenames include instance numbers)
    npy_files.sort(key=lambda x: int(os.path.splitext(os.path.basename(x))[0].split('_')[-1].replace("Instance", "")))

    all_histograms = []

    for file_path in npy_files:
        # Load the image data from the .npy file
        image_data = np.load(file_path)
        
        # Resize the image to 256x256
        resized_image = cv2.resize(image_data, (256, 256), interpolation=cv2.INTER_LINEAR)
        
        # Split the image into left and right hemispheres
        left_hemisphere = resized_image[:, :128]
        right_hemisphere = resized_image[:, 128:]
        
        # Exclude zeros and compute histograms
        left_hist, _ = np.histogram(left_hemisphere[left_hemisphere > 0], bins=num_bins, range=(0, 255))
        right_hist, _ = np.histogram(right_hemisphere[right_hemisphere > 0], bins=num_bins, range=(0, 255))
        
        # If no non-zero pixels are found, skip this slice
        if left_hist.sum() == 0 and right_hist.sum() == 0:
            print(f"Skipping slice with no non-zero pixels in file: {file_path}")
            continue
        
        # Combine histograms for left and right hemispheres
        combined_hist = np.concatenate([left_hist, right_hist])
        
        # Normalize histogram to ensure equal contribution of each slice
        if combined_hist.sum() > 0:
            combined_hist = combined_hist / np.sum(combined_hist)
        
        all_histograms.append(combined_hist)
    
    # If no valid histograms were found, return None
    if not all_histograms:
        print(f"No valid histograms found for the subject folder: {subject_folder}")
        return None
    
    # If there are more than 16 slices, take the first 16
    # If there are fewer than 16 slices, pad with zeros
    num_slices = len(all_histograms)
    if num_slices < 16:
        all_histograms.extend([np.zeros(2 * num_bins)] * (16 - num_slices))
    elif num_slices > 16:
        all_histograms = all_histograms[:16]
    
    # Flatten the list of histograms into a single feature vector
    subject_histogram_vector = np.concatenate(all_histograms)
    
    return subject_histogram_vector


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve, auc, precision_score, recall_score

In [4]:
class BrainHistogramDataset(Dataset):
    def __init__(self, base_folder, num_bins=255):
        self.num_bins = num_bins
        
        # List all subject folders in both 'normal' and 'abnormal' directories
        normal_folder = os.path.join(base_folder, 'normal')
        abnormal_folder = os.path.join(base_folder, 'abnormal')
        
        self.subject_folders = []
        self.labels = []
        
        # Add normal subjects
        normal_subject_folders = [os.path.join(normal_folder, f) for f in os.listdir(normal_folder) if os.path.isdir(os.path.join(normal_folder, f))]
        self.subject_folders.extend(normal_subject_folders)
        self.labels.extend([0] * len(normal_subject_folders))  # Label 0 for normal
     
        
        # Add abnormal subjects
        abnormal_subject_folders = [os.path.join(abnormal_folder, f) for f in os.listdir(abnormal_folder) if os.path.isdir(os.path.join(abnormal_folder, f))]
        self.subject_folders.extend(abnormal_subject_folders)
        self.labels.extend([1] * len(abnormal_subject_folders))  # Label 1 for abnormal

    def __len__(self):
        return len(self.subject_folders)

    def __getitem__(self, idx):
        subject_folder = self.subject_folders[idx]
        histogram_vector = compute_histograms_for_subject(subject_folder, self.num_bins)
        
        # Handle cases where histogram_vector is None
        if histogram_vector is None:
            histogram_vector = np.zeros(16 * 2 * self.num_bins)
        
        label = self.labels[idx]
        return torch.tensor(histogram_vector, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

In [5]:
# Paths to your data folders
train_folder = '/kaggle/working/final_data/train'
val_folder = '/kaggle/working/final_data/val'

In [6]:
# Create datasets
train_dataset = BrainHistogramDataset(train_folder)
val_dataset = BrainHistogramDataset(val_folder)


In [7]:
from sklearn.utils.class_weight import compute_class_weight

In [8]:
def compute_sample_weights(train_dataset):
    # Extract labels from the dataset
    labels = [label for _, label in train_dataset]
    
    # Calculate the number of samples per class
    class_sample_counts = np.bincount(labels)
    
    # Compute class weights as the inverse of the class sample counts
    class_weights = 1.0 / (class_sample_counts + 1e-9)  # Adding a small value to avoid division by zero
    
    # Normalize class weights (optional)
    class_weights = class_weights / class_weights.sum() * len(class_weights)
    print(class_weights)
    
    # Create sample weights based on class weights
    sample_weights = np.array([class_weights[label] for label in labels])
    
    # Scale the weights if they are too small
    sample_weights *= len(sample_weights)  # Scale by the number of samples
    
    return sample_weights

In [9]:
# Example usage:
sample_weights = compute_sample_weights(train_dataset)

[0.24759615 1.75240385]


In [10]:
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

In [11]:
train_loader = DataLoader(train_dataset, batch_size=64, sampler=sampler)

In [12]:
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [13]:
class MLPClassifier(nn.Module):
    def __init__(self, input_size):
        super(MLPClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.sigmoid(self.fc3(x))
        return x

In [23]:
def train_model(model, train_loader, val_loader, num_epochs=100, model_save_path='best_mlp_model.pth'):
    criterion = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Smaller learning rate
    best_auc_pr = 0.0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device).float()

            optimizer.zero_grad()
            outputs = model(inputs).squeeze(1)  # Ensure outputs are of shape (batch_size,)
        
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_loss = running_loss / len(train_loader)
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}')

        # Validation
        model.eval()
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device).float()
                outputs = model(inputs).squeeze(1)  # Ensure outputs are of shape (batch_size,)
                
                preds = (outputs > 0.5).cpu().numpy()  # Binarize predictions
                all_preds.extend(preds)
                all_labels.extend(labels.cpu().numpy())

        precision = precision_score(all_labels, all_preds, zero_division=1)
        recall = recall_score(all_labels, all_preds)
        auc_score = roc_auc_score(all_labels, all_preds)
        auc_pr = average_precision_score(all_labels, all_preds)

        print(f'Validation - Precision: {precision:.4f}, Recall: {recall:.4f}, AUC: {auc_score:.4f}, AUC-PR: {auc_pr:.4f}')

        # Save model if it's the best so far
        if auc_pr > best_auc_pr:
            best_auc_pr = auc_pr
            torch.save(model.state_dict(), model_save_path)
            print(f'Saved best model with AUC-PR: {best_auc_pr:.4f}')

    print('Training complete.')

In [24]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score, average_precision_score

In [ ]:
# Define the input size based on histogram size (16 slices * 2 hemispheres * 50 bins)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size = 16 * 2 * 255
model = MLPClassifier(input_size=input_size)

# Train the model
best_model = train_model(model, train_loader, val_loader, model_save_path='best_mlp_model.pth')

In [27]:
test_folder = '/kaggle/working/final_data/test'
test_dataset = BrainHistogramDataset(test_folder)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [32]:
model_test = MLPClassifier(input_size=input_size)
model_test.load_state_dict(torch.load('best_mlp_model.pth' ,weights_only=True))
model_test.to(device)
model_test.eval()

MLPClassifier(
  (fc1): Linear(in_features=8160, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.3, inplace=False)
  (sigmoid): Sigmoid()
)

In [ ]:
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device).float()
        outputs = model_test(inputs).squeeze(1)
        
        preds = (outputs > 0.5).cpu().numpy()  # Binarize predictions
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())

# Convert lists to numpy arrays for metric calculations
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Calculate metrics
precision = precision_score(all_labels, all_preds, zero_division=1)
recall = recall_score(all_labels, all_preds)
auc_score = roc_auc_score(all_labels, all_preds)
auc_pr = average_precision_score(all_labels, all_preds)

print(f'Test - Precision: {precision:.4f}, Recall: {recall:.4f}, AUC: {auc_score:.4f}, AUC-PR: {auc_pr:.4f}')